<a href="https://colab.research.google.com/github/hericax/Pos/blob/master/projeto_3/aed_projeto_3(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PARTE 2

# Projeto #3 - Meu primeiro projeto de IA

Antes de começar, leia as [Instruções](https://github.com/thvmm/pos-ds-ia/blob/master/projeto_3/README.md) e os [Critérios de Avaliação](https://github.com/thvmm/pos-ds-ia/blob/master/projeto_3/README.md).


### 4) **(30%)** Análise

Com seu dado pronto, é hora de fazer a parte mais legal: investigar e responder nossas hipoteses. Elas se confirmam? Em quais cenários? Existe alguma forma de perceber isso no mundo real? Dica: Abuse das técnicas de visualização.

Ex: Ainda no contexto dos exemplos do item 2). Ao investigar a hipotese relacionada ao padrão de compra, percebi que em todos os meses existia uma diferença entre dia util e final de semana. Porém notei que Fevereiro possui um comportamento diferente, o que poderia explicar? Talvez o carnaval e seus feriados.

Ex2: Talvez eu descubra que alguns finais de semana possuam um comportamento diferente dos dias de semana e outros não. Por que não são todos? Nessa investigação você pode acabar descobrindo que os finais de semana que são diferentes, coincidem no fato de serem os primeiros dias utéis, o que pode remeter ao fato de boa parte das empresas realizarem pagamentos nessa parte do mês.

Questionamentos a serem investigados:

1 - As maiores média de votos está relacionada a algum genero específico?

2 - As maiores média de votos está relacionada a algum idioma original?

3 - As maiores média de votos está relacionada a algum cia de produção?

!pip install graphviz==0.9
!pip install pydot
!pip install seaborn==0.9.0

!apt-get install graphviz

In [ ]:

# IMPORTANDO BIBLIOTECAS

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#import missingno as msno #! pip install missingno


from sklearn.preprocessing import StandardScaler #z-score
from sklearn import preprocessing # normalização MIM-MAX
from sklearn.preprocessing import RobustScaler 
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split # Divisão dos dados em treino e teste

from sklearn.tree import DecisionTreeRegressor
# from sklearn import tree
from sklearn.metrics import confusion_matrix


from sklearn.metrics import accuracy_score
from sklearn import utils

import multiprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import time

from sklearn.ensemble import RandomForestRegressor

In [ ]:
# lendo arquivo no google drive
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
arquivo_base = '/content/drive/My Drive/Colab Notebooks/projeto_3/filmes/filmes_df_limpo.csv'
df = pd.read_csv(arquivo_base)
df.head()
'''

In [ ]:
# lendo o arquivo localmente
df_final = pd.read_csv(r'C:\Users\User\Documents\Herica\pos\GitHub\Pos\projeto_3\base\filmes_df_limpo.csv')

In [ ]:
df_final.head(3)

In [ ]:
df_final.columns

## Hipótese 1 - As maiores média de votos está relacionada a algum genero específico?

In [ ]:
df_final.dtypes

In [ ]:
# Como são várias as features de gênero, as dividi em grupo para facilitar a visualização
x = df_final.groupby(['media_votos']).sum()

x['acao'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='Ação')
x['aventura'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Aventura')
x['animacao'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Animação')
x['comedia'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Comédia')
x['crime'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Crime')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que a comédia é o gênero com maior média de votos, seguido da ação

In [ ]:
# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['documentario'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='Documentário')
x['drama'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Drama')
x['familia'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Família')
x['fantasia'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Fantasia')
x['estrangeiro'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Estrangeiro')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o drama é o gênero com maior média de votos, estando os demais mais ou menos no memo nível

In [ ]:
# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['historia'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='História')
x['horror'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Horror')
x['musical'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Musical')
x['misterio'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Mistério')
x['romance'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Romance')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o romance é o gênero com maior média de votos, segido do horror.

In [ ]:
# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['ficcao_cientifica'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='Ficção')
x['filmes_tv'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Seriados')
x['suspense'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Suspense')
x['gerra'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Gerra')
x['ocidental'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Ocidental')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o suspense é o gênero com maior média de votos, seguido da ficção

In [ ]:
# plotando os maiores dos gêneros obtidos nos gráficos acima: comédia, suspense, romance e drama

# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['comedia'].plot(kind = 'line',color='pink',grid=True, stacked=True,label='Comédia')
x['suspense'].plot(kind = 'line',color='blue',grid=True, stacked=True,label='Suspense')
x['romance'].plot(kind = 'line',color='magenta',grid=True, stacked=True,label='Romance')
x['drama'].plot(kind = 'line',color='red', grid=True, stacked=True,label='Drama')

plt.legend()
plt.title('GÊneros x Média Votos')
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o drama é o gênero com maior média de votos, seguido da comédia. Os demais estão parecidos.

Conforme observamos, o gênero que possui a maior média de votos é o Drama, seja isolado, seja em conjunto com outros gêneros. Dos 10 maiores, o Drama está presente em 6. 

## Hipótese 2 - As maiores média de votos está relacionada a algum idioma original?

In [ ]:
df_final.columns

In [ ]:
lingua = df_final[['lingua','media_votos']].groupby('lingua').sum().sort_values(by = 'media_votos', ascending = False).head(10)
lingua

In [ ]:
# plotando os 10 idiomas originais com maior média de votos
x = df_final[['lingua','media_votos']].groupby('lingua').sum().sort_values(by = 'media_votos', ascending = False).head(10)
x = x.reset_index()

plt.bar(x.lingua,x.media_votos,color='red')


plt.xticks(rotation=75)
plt.xlabel("Lingua original")
plt.ylabel("Média de votos")
plt.title("Lingua x média de votos")

In [ ]:
total_ingles = len(df_final[df_final['lingua'] == "Inglês"])

outras_linguas = ['Francês', 'Chines', 'Italiano', 'Persa', 'Neerlandês',
       'Alemão', 'Mandarim padrão', 'Argentino', 'Espanhol', 'Russo',
       'Japonês', 'Coreano', 'Sérvio', 'Bengali', 'Hebraico', 'Português',
       'Wolof', 'Romeno', 'Húngaro', 'Galês', 'Vietnamita', 'hrv',
       'Dinamarquês', 'Norueguês', 'Bokmål norueguês', 'Polonês',
       'Grego, Moderno', 'Shan', 'No Language', 'Macedônio', 'Tibetano',
       'Catalão', 'Finlandês', 'Tailandês', 'Eslovaco', 'Bósnio', 'Hindi',
       'Turco', 'Islandês', 'Pachto', 'Abecásio', 'Esperanto',
       'Georgiano', 'Mongol', 'Bambara', 'Zulu', 'Ucraniano', 'Africâner',
       'Latim', 'Estoniano', 'Curdo', 'Letão', 'Tâmil', 'Esloveno',
       'Tagalog', 'Urdu', 'Kinyarwanda', 'Indonésio', 'Búlgaro',
       'Marathi (Marāṭhī)', 'Lituano', 'Cazaque', 'Malaio', 'Albanês',
       'Quechua', 'Telugu', 'Amárico', 'Javanês', 'Tajique', 'Malayalam',
       'Croata', 'Lao', 'Aymará', 'Canarês', 'Nepali', 'Panjabi, Punjabi',
       'Galego', 'Quirguiz, Kyrgyz', 'Samoano', 'Basco', 'Tcheco',
       'Armênio', 'Inuktitut', 'Sinhala, Cingalês']
total_outras = 0
for i in outras_linguas:
    total_outras = total_outras + len(df_final[df_final['lingua'] == i])

total_linguas = total_outras + total_ingles

In [ ]:
print ('total_outras=',total_outras, 'total_ingles=', total_ingles, '\n Total linguas', total_linguas, '\n diferença ingles - outras= ', total_ingles - total_outras, '\n registros dataset=', len(df_final.lingua))

In [ ]:
print ("Ingles=", (total_ingles/total_linguas)*100)
print ("Outras=", (total_outras/total_linguas)*100)

Percebe-se acima que mesmo somando todas as outras linguas originais dos filmes, o inglês é o maior idioma, com 70%. 

## Hipótese 3 - As maiores média de votos está relacionada a algum cia de produção?

In [ ]:
df_final.columns

In [ ]:
print ('MGM', len(df_final[df_final['Metro-Goldwyn-Mayer (MGM)'] == True]))
print ('Warner', len(df_final[df_final['Warner Bros.'] == True]))
print ('Paramount', len(df_final[df_final['Paramount Pictures'] == True]))
print ('Fox', len(df_final[df_final['Twentieth Century Fox Film Corporation'] == True]))
print ('Universal', len(df_final[df_final['Universal Pictures'] == True]))
print ('RKO', len(df_final[df_final['RKO Radio Pictures'] == True]))
print ('Columbia', len(df_final[df_final['Columbia Pictures'] == True]))
print ('Mosfilm', len(df_final[df_final['Mosfilm'] == True]))
print ('Walt Disney', len(df_final[df_final['Walt Disney Pictures'] == True]))
print ('Universal International Pictures (UI)', len(df_final[df_final['Universal International Pictures (UI)'] == True]))

In [ ]:
# plotando os maiores dos gêneros obtidos nos gráficos acima: comédia, suspense, romance e drama

# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['Metro-Goldwyn-Mayer (MGM)'].plot(kind = 'line',color='pink',grid=True, stacked=True,label='MGM')
x['Warner Bros.'].plot(kind = 'line',color='blue',grid=True, stacked=True,label='Warner Bros.')
x['Paramount Pictures'].plot(kind = 'line',color='magenta',grid=True, stacked=True,label='Paramount Pictures')
x['Twentieth Century Fox Film Corporation'].plot(kind = 'line',color='red', grid=True, stacked=True,label='Twentieth Century Fox Film Corporation')
x['Universal Pictures'].plot(kind = 'line',color='yellow', grid=True, stacked=True,label='Universal Pictures')
x['RKO Radio Pictures'].plot(kind = 'line',color='brown', grid=True, stacked=True,label='RKO Radio Pictures')
x['Columbia Pictures'].plot(kind = 'line',color='black', grid=True, stacked=True,label='Columbia Pictures')
x['Mosfilm'].plot(kind = 'line',color='green', grid=True, stacked=True,label='Mosfilm')
x['Walt Disney Pictures'].plot(kind = 'line',color='purple', grid=True, stacked=True,label='Walt Disney Pictures')
x['Universal International Pictures (UI)'].plot(kind = 'line',color='grey', grid=True, stacked=True,label='Universal International Pictures (UI)')

plt.legend()
plt.title('Companias x Média Votos')
plt.xlabel("Companias")
plt.ylabel("Média Votos")

In [ ]:
df_final.columns

In [ ]:
# plotando os maiores dos gêneros obtidos nos gráficos acima: comédia, suspense, romance e drama

# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()


x['Metro-Goldwyn-Mayer (MGM)'].plot(kind = 'bar',color='pink',grid=True, stacked=True,label='MGM')
x['Warner Bros.'].plot(kind = 'bar',color='blue',grid=True, stacked=True,label='Warner Bros.')
x['Paramount Pictures'].plot(kind = 'bar',color='magenta',grid=True, stacked=True,label='Paramount Pictures')
x['Twentieth Century Fox Film Corporation'].plot(kind = 'bar',color='red', grid=True, stacked=True,label='Twentieth Century Fox Film Corporation')
x['Universal Pictures'].plot(kind = 'bar',color='yellow', grid=True, stacked=True,label='Universal Pictures')
x['RKO Radio Pictures'].plot(kind = 'bar',color='brown', grid=True, stacked=True,label='RKO Radio Pictures')
x['Columbia Pictures'].plot(kind = 'bar',color='black', grid=True, stacked=True,label='Columbia Pictures')
x['Mosfilm'].plot(kind = 'bar',color='green', grid=True, stacked=True,label='Mosfilm')
x['Walt Disney Pictures'].plot(kind = 'bar',color='purple', grid=True, stacked=True,label='Walt Disney Pictures')
x['Universal International Pictures (UI)'].plot(kind = 'bar',color='grey', grid=True, stacked=True,label='Universal International Pictures (UI)')


plt.legend()
plt.title('Companias x Média Votos')
plt.xlabel("Companias")
plt.ylabel("Média Votos")

Percebe-se que a companhia com as maiores médias de votos é a Warner

### 5) Modelagem 30%

Agora você terá mais uma oportunidade de mostrar o que você aprendeu durante o módulo 2 quanto a modelagem de dados e criação de modelos. Utilizando os dados preparados na seção 1 e após a análise feita na seção 2 você deverá:
1. **Defina um problema de regressão ou classificação que envolva uma variável dos seus dados.**

Será criado um modelo de regrassão para prever a média de votos dos filmes.

2. **Realize ao menos 2 técnicas de processamento e seleção de features.**
    * Isto inclui, normalização, PCA, e técnicas de seleção de features como information gain. Seja criativo pois está parte é crucial para seu modelo. Você pode escolhar manualmente as features desde que seja justificada na seção 2 (parte de Análise).


### DISCRETIZAÇÃO DA FEATURE DURACAO

In [ ]:
# verificando a correlação da duração com a média de votos
corr_ini_duracao = df_final['duracao'].corr(df_final['media_votos'])
print ('duração x media_votos',corr_ini_duracao)

In [ ]:
df_final.duracao.dtype

In [ ]:
# Vou tentar criar grupos para definir a feture e verificar se há melhora na correlação
intervals = pd.qcut(df_final['duracao'], 2).astype(str).unique()
intervals

In [ ]:
df_final['tipo_duracao'] = pd.qcut(df_final['duracao'], 2).astype(str).map( {
                            intervals[0]: 'media',
                            intervals[1]: 'longa'
                            } ).astype(str)
df_final[['duracao','tipo_duracao']]

In [ ]:
# plotando os 10 idiomas originais com maior média de votos
x = df_final[['tipo_duracao','media_votos']].groupby('tipo_duracao').sum().sort_values(by = 'media_votos', ascending = False).head(10)
x = x.reset_index()

plt.bar(x.tipo_duracao,x.media_votos,color='red')


plt.xticks(rotation=75)
plt.xlabel("Tipo de duração")
plt.ylabel("Média de votos")
plt.title("Tipo de duração x média de votos")

In [ ]:
# Convertando strings em numéricos usando dicionário

df_final['tipo_duracao'] = df_final['tipo_duracao'].map({
'media': 0, 
'longa': 1})

In [ ]:
# verificando a correlação da duração com a média de votos
corr_fim_duracao = df_final['tipo_duracao'].corr(df_final['media_votos'])
print ('Tipo duração x media_votos',corr_fim_duracao)

In [ ]:
print ('Final', corr_fim_duracao, ' | inicial', corr_ini_duracao)

Fiz o teste da correlação para os valores de 2 a 5 grupos. Quanto maior a qtde de grupos, pior a correlação.
Nova feature mantida pois houve uma pequena melhora na correlação.

### DISCRETIZAÇÃO DA FEATURE ANO_LANCAMENTO

In [ ]:
# verificando a correlação da duração com a média de votos
corr_ini_anoLancamento = df_final['ano_lancamento'].corr(df_final['media_votos'])
print ('Ano lançamento x media_votos',corr_ini_anoLancamento)

In [ ]:
# tentar criar períodos para melhor a correlação
# avaliando a quantidade de períodos a serem criados 
intervals = pd.qcut(df_final['ano_lancamento'], 10).astype(str).unique()
intervals

In [ ]:
# criando os perídos para definir o ano de lançamento dos filmes
df_final['periodo'] = pd.qcut(df_final['ano_lancamento'], 10).astype(str).map({intervals[0]: '1993 a 2000',
        intervals[1]: '1970 a 1982',
        intervals[2]: '1982 a 1993',
        intervals[3]: '1952 a 1970',
        intervals[4]: '1873 a 1952',
        intervals[5]: '2000 a 2005',
        intervals[6]: '2009 a 2012',
        intervals[7]: '2012 a 2014',
        intervals[8]: '2005 a 2009',
        intervals[9]: '2014 a 2020'} ).astype(str)

df_final[['ano_lancamento','periodo']]

In [ ]:
# plotando os 10 idiomas originais com maior média de votos
x = df_final[['periodo','media_votos']].groupby('periodo').sum().sort_values(by = 'media_votos', ascending = False).head(10)
x = x.reset_index()

plt.bar(x.periodo,x.media_votos,color='red')


plt.xticks(rotation=75)
plt.xlabel("Período")
plt.ylabel("Média de votos")
plt.title("Período x média de votos")

In [ ]:
# Convertando strings em numéricos usando dicionário

df_final['tipo_duracao'] = df_final['periodo'].map({
    '1993 a 2000': 0,
    '1970 a 1982': 1,
    '1982 a 1993': 2,
    '1952 a 1970': 3,
    '1873 a 1952': 4,
    '2000 a 2005': 5,
    '2009 a 2012': 6,
    '2012 a 2014': 7,
    '2005 a 2009': 8,
    '2014 a 2020': 9
})

In [ ]:
# verificando a correlação da duração com a média de votos
corr_fim_anoLancamento = df_final['tipo_duracao'].corr(df_final['media_votos'])
print ('Ano lançamento x media_votos',corr_fim_anoLancamento)

In [ ]:
print (" final= ", corr_fim_anoLancamento, "| inicial = ",corr_ini_anoLancamento)

In [ ]:
# Houve melhora então a nova feature será mantida

### FAZER A CORRELAÇÃO DAS FEATURES A SEREM TRABALHADAS

In [ ]:
# verificando a correlação dos generos

print ('acao x media_votos',df_final['acao'].corr(df_final['media_votos']))
print ('aventura x media_votos',df_final['aventura'].corr(df_final['media_votos']))
print ('animacao x media_votos',df_final['animacao'].corr(df_final['media_votos']))
print ('comedia x media_votos',df_final['comedia'].corr(df_final['media_votos']))
print ('crime x media_votos',df_final['crime'].corr(df_final['media_votos']))
print ('documentario x media_votos',df_final['documentario'].corr(df_final['media_votos']))
print ('drama x media_votos',df_final['drama'].corr(df_final['media_votos']))
print ('familia x media_votos',df_final['familia'].corr(df_final['media_votos']))
print ('fantasia x media_votos',df_final['fantasia'].corr(df_final['media_votos']))
print ('estrangeiro x media_votos',df_final['estrangeiro'].corr(df_final['media_votos']))
print ('historia x media_votos',df_final['historia'].corr(df_final['media_votos']))
print ('horror x media_votos',df_final['horror'].corr(df_final['media_votos']))
print ('musical x media_votos',df_final['musical'].corr(df_final['media_votos']))
print ('misterio x media_votos',df_final['misterio'].corr(df_final['media_votos']))
print ('romance x media_votos',df_final['romance'].corr(df_final['media_votos']))
print ('ficcao_cientifica x media_votos',df_final['ficcao_cientifica'].corr(df_final['media_votos']))
print ('filmes_tv x media_votos',df_final['filmes_tv'].corr(df_final['media_votos']))
print ('suspense x media_votos',df_final['suspense'].corr(df_final['media_votos']))
print ('gerra x media_votos',df_final['gerra'].corr(df_final['media_votos']))
print ('ocidental x media_votos',df_final['ocidental'].corr(df_final['media_votos']))

In [ ]:
#Excluir as features com correlação negativa
'''acao x media_votos -0.08289026991958133
aventura x media_votos -0.02794159637963377
comedia x media_votos -0.01778207543223205
familia x media_votos -0.01386665979818757
fantasia x media_votos -0.013615448225225833
estrangeiro x media_votos -0.0045234706117887975
horror x media_votos -0.18819159408147684
misterio x media_votos -0.008024703581022141
ficcao_cientifica x media_votos -0.11295899314774636
filmes_tv x media_votos -0.03709269978090582
suspense x media_votos -0.09543228084790208
ocidental x media_votos -0.03503105661478576'''

df_final.drop(['acao','aventura','comedia', 'familia', 'fantasia', 'estrangeiro', 'horror', 'misterio',
    'ficcao_cientifica', 'filmes_tv', 'suspense', 'ocidental'], axis = 1, inplace = True)

In [ ]:
print ('video x media_votos',df_final['video'].corr(df_final['media_votos']))

In [ ]:
# remoção da feature video
df_final.drop(['video'], axis = 1, inplace = True)

In [ ]:
df_final.columns

In [ ]:
# features de companhias de produção
''''Metro-Goldwyn-Mayer (MGM)','Warner Bros.', 'Paramount Pictures',
       'Twentieth Century Fox Film Corporation', 'Universal Pictures',
       'RKO Radio Pictures', 'Columbia Pictures', 'Mosfilm',
       'Walt Disney Pictures', 'Universal International Pictures (UI)'
       '''

print ('MGV',df_final['Metro-Goldwyn-Mayer (MGM)'].corr(df_final['media_votos']))
print ('Warner Bros. x media_votos',df_final['Warner Bros.'].corr(df_final['media_votos']))
print ('Paramount x media_votos',df_final['Paramount Pictures'].corr(df_final['media_votos']))
print ('Fox x media_votos',df_final['Twentieth Century Fox Film Corporation'].corr(df_final['media_votos']))
print ('Universal Pictures x media_votos',df_final['Universal Pictures'].corr(df_final['media_votos']))
print ('RKO Radio Pictures x media_votos',df_final['RKO Radio Pictures'].corr(df_final['media_votos']))
print ('Columbia Pictures x media_votos',df_final['Columbia Pictures'].corr(df_final['media_votos']))
print ('Mosfilm x media_votos',df_final['Mosfilm'].corr(df_final['media_votos']))
print ('Walt Disney Pictures x media_votos',df_final['Walt Disney Pictures'].corr(df_final['media_votos']))
print ('Universal International Pictures (UI) x media_votos',df_final['Universal International Pictures (UI)'].corr(df_final['media_votos']))

In [ ]:
# excluido das features com correlação negativa
'''
MGV -0.005643144906661703
RKO Radio Pictures x media_votos -0.005677699693419887

'''

df_final.drop(['Metro-Goldwyn-Mayer (MGM)','RKO Radio Pictures'], axis = 1, inplace = True)

In [ ]:
# features de paizes onde os filmes foram produzidos
'''
'United States of America', 'United Kingdom', 'France', 'Japan',
       'Italy', 'Canada', 'Canada.1', 'Canada.2', 'Canada.3', 'Germany',
       'Russia', 'India', 'Canada.4', 'Canada.5', 'Canada.6', 'Canada.7',
       'English', 'Français', '日本語', 'Italiano', 'Español'
    '''
print ('United States of America',df_final['United States of America (pais producao)'].corr(df_final['media_votos']))
print ('United Kingdom x media_votos',df_final['United Kingdom (pais producao)'].corr(df_final['media_votos']))
print ('France x media_votos',df_final['France (pais producao)'].corr(df_final['media_votos']))
print ('Japan x media_votos',df_final['Japan (pais producao)'].corr(df_final['media_votos']))
print ('Italy x media_votos',df_final['Italy (pais producao)'].corr(df_final['media_votos']))
print ('Canada x media_votos',df_final['Canada (pais producao)'].corr(df_final['media_votos']))


In [ ]:
# remoção das features com correlação negativa
'''
United States of America -0.10648717484568719
Canada x media_votos -0.04293418732490301
'''

df_final.drop(['United States of America (pais producao)','Canada (pais producao)'], axis = 1, inplace = True)

In [ ]:
df_final.columns

In [ ]:
# correlação dos idimos de tradução
''' 'English', 'Français', '日本語', 'Italiano', 'Español' '''

print ('English x media_votos',df_final['English'].corr(df_final['media_votos']))
print ('Français x media_votos',df_final['Français'].corr(df_final['media_votos']))
print ('日本語 x media_votos',df_final['日本語'].corr(df_final['media_votos']))
print ('Italiano x media_votos',df_final['Italiano'].corr(df_final['media_votos']))
print ('Español x media_votos',df_final['Español'].corr(df_final['media_votos']))

In [ ]:
# remoção das features com correlação negativa
'''
English x media_votos -0.1104885381792159
'''

df_final.drop(['English'], axis = 1, inplace = True)

# inicio dos testes

In [ ]:
df_final.columns

Dividindo o target em classes

In [ ]:
# dividindo em três classes
pd.cut(df_final['media_votos'], 5).head()

In [ ]:
df_final['target'] = pd.cut(df_final['media_votos'], 5)

In [ ]:
df_final['target']

In [ ]:
plt.style.use('ggplot')

a = df_final.groupby('target').size()

categories = df_final['target'].cat.categories
ind = np.array([x for x, _ in enumerate(categories)])
width = 0.35       
plt.bar(ind, a, width, label='target')

plt.xticks(ind + width / 3, categories)
plt.legend(loc='best')
plt.xticks(rotation = 90)
plt.show()

# fim dos testes

In [ ]:
df_final.columns

## SELECIONANDO AS FEATURES

In [ ]:
#Selecionando as features que serão utilizadas para redução da dimensionalidade e normalização 
X = df_final[[ 
    'ano_lancamento', 'cod_lingua', 'qtde_votos', 
       'animacao', 'crime',
       'documentario', 'drama', 'historia', 'musical', 'romance', 'gerra',
       'Warner Bros.', 'Paramount Pictures',
       'Twentieth Century Fox Film Corporation', 'Universal Pictures',
       'Columbia Pictures', 'Mosfilm', 'Walt Disney Pictures',
       'Universal International Pictures (UI)',
       'United Kingdom (pais producao)', 'France (pais producao)',
       'Japan (pais producao)', 'Italy (pais producao)',
       'Canada (pais producao).1', 'Germany (pais producao)',
       'Russia (pais producao)', 'India (pais producao)', 'Français', '日本語',
       'Italiano', 'Español', 'tipo_duracao'
       ]]
       
y = df_final['media_votos'].round(0)

In [ ]:
y = y.astype(int)

In [ ]:
# target desbalanceados
y.value_counts()

In [ ]:
# guardando o rótulo das colunas
cols = X.columns

## REDUÇÃO DA DIMENSIONALIDADE



In [ ]:
# AVALIAR QUAL A QUANTIDADE DE COMPONENTES UTILIZAR NO pca

plt.rcParams['figure.figsize'] = [12, 5]

pca = PCA().fit(X)
plt.plot(np.cumsum(pca.explained_variance_ratio_), color='#191970')

plt.title('Escolha do número de componentes', y=1.03, size=14, loc='left', x=-0.06)
plt.ylabel('Variância explicada cumulativa', horizontalalignment='left', y=0.30)
plt.xlabel('Número de componentes', horizontalalignment='right', x=0.265)
plt.show()

In [ ]:
# APLICAR O PCA
dados_dimensionados = PCA(n_components=3).fit_transform(X)

dados_dimensionados = pd.DataFrame(dados_dimensionados)

dados_dimensionados.head(3)

In [ ]:
# APLICAR O PCA
'''
dados_dimensionados = PCA(n_components=3).fit_transform(X)
dados_dimensionados
'''

In [ ]:
dados_dimensionados.shape

## NORMALIZAÇÃO DOS DADOS

NORMALIZAÇÃO MIM-MAX

In [ ]:
# A normalização é feita feature a feature e o objeto é colocar as variáveis dentro do intervalo de 0 e 1
min_max_scaler = preprocessing.MinMaxScaler() #dimensiona e traduz cada recurso individualmente, de modo que esteja no intervalo especificado no conjunto de treinamento (entre zero e um)
x_scaled = min_max_scaler.fit_transform(X)
df_normalizado_MIMMAX = pd.DataFrame(x_scaled)

# renomeando as colunas
df_normalizado_MIMMAX.set_axis(cols, axis='columns', inplace=True)
df_normalizado_MIMMAX.head(3)

In [ ]:
df_normalizado_MIMMAX.shape


NORMALIZAÇÃO Z-SCORE

In [ ]:
# normalizar os recursos (cada coluna dos dados) para que cada coluna/recurso/variável tenha mean = 0 e standard deviation = 1.
scaler = StandardScaler()
df_normalizado_scaler = scaler.fit_transform(X)
df_normalizado_scaler = pd.DataFrame(df_normalizado_scaler)

# renomeando as colunas
df_normalizado_scaler.set_axis(cols, axis='columns', inplace=True)
df_normalizado_scaler.head(3)

In [ ]:
df_normalizado_scaler.shape

In [ ]:
type(df_normalizado_scaler)

RobustScaler

In [ ]:
#subtrai a média do valor em questão e então divide o resultado pelo segundo quartil.
df_nomealizado_robust = RobustScaler().fit_transform(X)
df_nomealizado_robust = pd.DataFrame(df_nomealizado_robust)

# renomeando as colunas
df_nomealizado_robust.set_axis(cols, axis='columns', inplace=True)
df_nomealizado_robust.head(3)

In [ ]:
df_nomealizado_robust.shape

3. **Defina uma métrica para avaliar o seu modelo.**
    * Por exemplo, você pode utilizar MAE (Mean Absolute Error) para um problema de regressão. Ou, F1-Score para um problema de classificação. Há varias métricas, então escolha sabiamente.


Será utilizado como métrica o desvio padrão, que que serve para dizer o quanto os valores dos quais se extraiu a média são próximos ou distantes da própria média. Assim, quando se calcula o desvio padrão juntamente com a média de diferentes grupos, obtém-se mais informações para avaliar e diferenciar seus comportamentos.

4. **Divida seus dados em 2 conjuntos. Um de treino e outro conjunto de teste.**
    * Treine e otimize seu modelo no conjunto de treino e apenas use o conjunto de teste para apresentar os resultados finais.


In [ ]:
# semente aleatória a ser utilizada em tydi
RANDOM_STATE = 1 #semente aleatória
test_size = 0.35 # 35% será amostra de teste

Divisão dos dados - redimencionados (PCA)

In [ ]:
# o parâmetro test_size indica o percentual de amostras que serão selecionadas para teste

X_train_PCA, X_test_PCA, y_train_PCA, y_test_PCA = train_test_split(dados_dimensionados,y, 
                                                    test_size=test_size, 
                                                    random_state=RANDOM_STATE)
                            

In [ ]:
print ('Quantidade de registros \n')
print('Dados de treino PCA: X =', X_train_PCA.shape,'y=', y_train_PCA.shape,'\n','Dados de teste: X= ', X_test_PCA.shape,'y=', y_test_PCA.shape )

Divisão dos dados - normalização MIM_MAx

In [ ]:
# o parâmetro test_size indica o percentual de amostras que serão selecionadas para teste

X_train_MIMMAX, X_test_MIMMAX, y_train_MIMMAX, y_test_MIMMAX = train_test_split(df_normalizado_MIMMAX, y, 
                                                    test_size=test_size, 
                                                    random_state=RANDOM_STATE)

In [ ]:
print ('Quantidade de registros \n')
print('Dados de treino PCA: X =', X_train_MIMMAX.shape,'y=', y_train_MIMMAX.shape,'\n','Dados de teste: X= ', X_test_MIMMAX.shape,'y=', y_test_MIMMAX.shape )

Divisão dos dados - normalização Z-SCORE

In [ ]:
# o parâmetro test_size indica o percentual de amostras que serão selecionadas para teste

X_train_scaler, X_test_scaler, y_train_scaler, y_test_scaler = train_test_split(df_normalizado_scaler, y, 
                                                    test_size=test_size, 
                                                    random_state=RANDOM_STATE)

In [ ]:
print ('Quantidade de registros \n')
print('Dados de treino PCA: X =', X_train_scaler.shape,'y=', y_train_scaler.shape,'\n','Dados de teste: X= ', X_test_scaler.shape,'y=', y_test_scaler.shape )

Divisão dos dados - normalização ROBUST

In [ ]:
# o parâmetro test_size indica o percentual de amostras que serão selecionadas para teste

X_train_robust, X_test_robust, y_train_robust, y_test_robust = train_test_split(df_nomealizado_robust, y, 
                                                    test_size=test_size, 
                                                    random_state=RANDOM_STATE)

In [ ]:
print ('Quantidade de registros \n')
print('Dados de treino PCA: X =', X_train_robust.shape,'y=', y_train_robust.shape,'\n','Dados de teste: X= ', X_test_robust.shape,'y=', y_test_robust.shape)

5. **Treine um ou mais modelos de ML para o seu problema.**
    * Escolha 1 ou mais tipos de classificadores ou regressores dependendo do seu problema.
    * Por exemplo, TreeClassifier para um problema de classificação. 
    * Use cross-validation e outras técnicas como GridSearch e ou RandomizedSearch para encontrar os melhores parametros para o seu modelo.


In [ ]:
'''
# criar o DF para armazenas as métrica coletadas em cada algoritmo
COLUNAS = [
    'tipo_dado', # se é PCA, MIM-MAX, Scaler ou robust
    'media_teste', 
    'media_treino',
    'diferenca_media',
    'desvio_teste' ,
    'desvio_teste',
    'diferenca_desvio'    
]

df_tree_metricas = pd.DataFrame(columns=COLUNAS)
df_tree_metricas
'''

# DEFININDO AS FUNÇÕES QUE SERÃO UTILIZADAS PARA RODAR OS MODELOS, PLOTAR A MATRIZ DE CONFUSÃO E ARMAZENAR AS MÉTRICAS COLETADAS

In [ ]:
# criando do dataframe para armazenar as métricas coletadas a cada execução do modelo
columns = ['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'] 
''''
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
metricas = pd.DataFrame(columns=columns)

# função para inserir registro no dataframe
def insert(df, row):
    insert_loc = df.index.max()

    if np.isnan(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row

In [ ]:
import itertools


# FUNÇÃO PARA RODAR O MODELO
def roda_modelo (tipo, nome_modelo, X_train, y_train, X_test, y_test, modelo):
    modelo.fit(X_train, y_train) # TREINANDO A ARVORE COM OS DADOS DE TREINO
    y_pred = modelo.predict(X_test) # predizendo o target
    
    print ("{} do {}".format(tipo, nome_modelo))
    print ("\n")

    classes = [0,1,2,3,4,5,6,7,8,9,10]
    resultados = mostra_resultados(y_test, y_pred, classes,tipo, nome_modelo)
    
    return resultados, modelo


# FUNÇÃO PARA PLOTAR A MATRIZ DE CONFUSÃO
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.grid(False)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# FUNÇÃO PARA MOSTRA RESULTADOS OBTIDOS NO MODELO
def mostra_resultados(y, y_pred, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.coolwarm):

    # Calcular os resultados
    #cm = confusion_matrix(y, y_pred.round(0))
    cm = confusion_matrix(y, y_pred)
    FP = cm.sum(axis=0) - np.diag(cm)  # Falso Positivos
    FN = cm.sum(axis=1) - np.diag(cm)  # Falso Negativos
    TP = np.diag(cm)                   # Verdadeiro Positivo
    TN = cm.sum() - (FP + FN + TP)     # Verdadeiro Negativos
    TPR = TP / (TP + FN)  # Revocação    
    PPV = TP / (TP + FP)  # Precisao
    F1 = 2 * ((PPV * TPR) / (PPV + TPR)) # F1 Score
    ACC = (TP + TN) / (TP + FP + FN + TN)  # Acurácia

    desvio_y_test = y.std()
    desvio_y_pred = y_pred.std()
    diferenca_desvio = desvio_y_test - desvio_y_pred

    media_y_test = y.mean()
    media_y_prev = y_pred.mean()
    diferenca_media = media_y_test - media_y_prev

    # Exibe resultados
    print("    Verdadeiros Positivos:{}".format(TP))
    print("    Verdadeiros Negativos:{}".format(TN))
    print("    Falso Positivo:{}".format(FP))
    print("    Falso Negativo:{}".format(FN))
    
    print("    Revocação:{}".format(TPR))    
    print("    Precisao:{}".format(PPV))
    
    
    print("\n\n")

    print("    F1 Score:{}".format(F1.round(4)))
    print("    \nAcuracia:{}".format(ACC.round(4)*100))

    print("    \n Desvio \n")
    print("    y_tes:{}".format(desvio_y_test))
    print("    y_pred:{}".format(desvio_y_pred))
    print("    Diferença:{}".format(diferenca_desvio))
    
    print("    \n Média \n")
    print("    y_tes:{}".format(media_y_test))
    print("    y_pred:{}".format(media_y_prev))
    print("    Diferença:{}".format(diferenca_media))
    
    print("\n\n")

    plot_confusion_matrix(cm, classes)

    return desvio_y_test, desvio_y_pred, diferenca_desvio, media_y_test, media_y_prev, diferenca_media, y_pred

# MODELO 1 - DECISION TREE

In [ ]:
# CRIAÇÃO DO MODELO COM PARÂMETROS PADRÃO
dec_clf = DecisionTreeRegressor(random_state=RANDOM_STATE)

In [ ]:
dec_clf

# INICIO TESTES

## PCA

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_PCA_tree = roda_modelo('BASELINE DECISION TREE', 'PCA', X_train_PCA, y_train_PCA, X_test_PCA, y_test_PCA, dec_clf)

#resultados_PCA = roda_arvore('BASELINE', 'PCA', X_train_PCA, y_train_PCA, X_test_PCA, y_test_PCA)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['DECISION TREE REGRESSOR', 'BASELINE', 'PCA', resultados_PCA_tree[0][0],resultados_PCA_tree[0][1], resultados_PCA_tree[0][2], resultados_PCA_tree[0][3],resultados_PCA_tree[0][4],resultados_PCA_tree[0][5]])

## MIM-MAX

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_MIMMAX_tree = roda_modelo('BASELINE DECISON TREE', 'MIM-MAX', X_train_MIMMAX, y_train_MIMMAX, X_test_MIMMAX, y_test_MIMMAX, dec_clf)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['DECISION TREE REGRESSOR', 'BASELINE', 'MIM-MAX', resultados_MIMMAX_tree[0][0],resultados_MIMMAX_tree[0][1], resultados_MIMMAX_tree[0][2], resultados_MIMMAX_tree[0][3],resultados_MIMMAX_tree[0][4],resultados_MIMMAX_tree[0][5]])

## Z-SCORE

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_scaler_tree = roda_modelo('BASELINE - DECISION TREE', 'SCALER', X_train_scaler, y_train_scaler, X_test_scaler, y_test_scaler, dec_clf)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['DECISION TREE REGRESSOR', 'BASELINE', 'SCALER', resultados_scaler_tree[0][0],resultados_scaler_tree[0][1], resultados_scaler_tree[0][2], resultados_scaler_tree[0][3],resultados_scaler_tree[0][4],resultados_scaler_tree[0][5]])

## ROBUST

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_robust_tree = roda_modelo('BASELINE DECISION TREE', 'ROBUST', X_train_robust, y_train_robust, X_test_robust, y_test_robust, dec_clf)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['DECISION TREE REGRESSOR', 'BASELINE', 'ROBUST', resultados_robust_tree[0][0],resultados_robust_tree[0][1], resultados_robust_tree[0][2], resultados_robust_tree[0][3],resultados_robust_tree[0][4],resultados_robust_tree[0][5]])

In [ ]:
metricas

In [ ]:
# conforme observado o modelo com o menor desvio foi o DECISION TREE REGRESSOR com a normalização MIM-MAX
print ("Modelo com menor desvio", min(metricas.diferenca_desvio))

# FIM TESTES

# inicio - deletar após o termino dos testes

## Baseline - PCA

In [ ]:
from sklearn import tree
tree.export_graphviz(resultados_robust_tree[1])

In [ ]:
digraph G {

  subgraph cluster_0 {
    style=filled;
    color=lightgrey;
    node [style=filled,color=white];
    a0 -> a1 -> a2 -> a3;
    label = "process #1";
  }

  subgraph cluster_1 {
    node [style=filled];
    b0 -> b1 -> b2 -> b3;
    label = "process #2";
    color=blue
  }
  start -> a0;
  start -> b0;
  a1 -> b3;
  b2 -> a3;
  a3 -> a0;
  a3 -> end;
  b3 -> end;

  start [shape=Mdiamond];
  end [shape=Msquare];
}

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(resultados_robust_tree[1], out_file=None, filled=True, rounded=True)
graph = graphviz.Source(dot_data)
graph

# fim - deletar após o termino dos testes

## Utilizando cross-validation e GridSearchCV para melhorar os modelos

In [ ]:
# obtendo quais são os parâmetros disponíveis na árvore, para uso no grid-search
dec_clf.get_params().keys()

In [ ]:
# parâmetros a serem avaliados
# Escolhi os parâmetros que acredito que são os mais comuns de serem utilizados
param_grid = [
              {
                 'criterion':['mse', 'friedman_mse', 'mae'],
                 'max_depth': [5, 20, 100, 400, None], # profundidade máxima da árvore
                 'min_samples_split': [2, 50, 300, 1000], # O número mínimo de amostras necessárias para dividir um nó interno
                 'min_samples_leaf': [30, 120, 420, 820], # número mínimo de amostras necessárias para estar em um nó folha
                 'random_state': [RANDOM_STATE], 
                 'splitter': ['best']
              }

]

In [ ]:
# execução do gridsearch para definição dos melhores parâmetros
gs = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=multiprocessing.cpu_count())
gs

# função para rodar o CROSS-VALIDATION

In [ ]:
# função que executa o griserachCV com a lista de parâmetros para definir a configuração com melhor score
def definir_parametros (X_train, y_train, gs):
    #classes = [0,1,2,3,4,5,6,7,8,9,10]
    #gs = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param_grid, scoring='accuracy', cv=15, n_jobs=multiprocessing.cpu_count())

    gs.fit(X_train, y_train) # TREINARNDO A ÁRVORE
    # execução do cross-validation com os parâmetros sugeridos pelo gridsearch
    cross_val_score(gs.best_estimator_,X_train, y_train,scoring='r2',cv=15).mean()
    
    
    print('Melhor estimador=',gs.best_estimator_)
    print("Melhor score=",gs.best_score_ * 100)

    return gs.best_estimator_

# INICIO DOS TESTES

## PCA

In [ ]:
y_train_PCA.value_counts()

In [ ]:
# ----------------------------------------
# Chama a função que roda o griSerachCV com os dados de treino para obter o melhor estimador
tic = time.time()

resultados_PCA_CV = definir_parametros(X_train_PCA, y_train_PCA,gs)

tac = time.time()
# ----------------------------------------

# TREINANDO A ARVORE COM OS DADOS DE TREINO
dec_clf_cv.fit(X_train_PCA, y_train_PCA) 
tempo_que_passou = tac - tic

# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_PCA_treecv = roda_modelo('GRIDSEARCHCV + CROSSVALIDATION - DECISION TREE', 'PCA', X_train_PCA, y_train_PCA, X_test_PCA, y_test_PCA, dec_clf_cv)

# impressão do tempo gasto no treino
print("Tempo %.2f segundos" % tempo_que_passou)

In [ ]:
# Armazenando os dados
insert(metricas,['DECISION TREE REGRESSOR', 'GS + CV', 'PCA', resultados_PCA_treecv[0][0],resultados_PCA_treecv[0][1], resultados_PCA_treecv[0][2], resultados_PCA_treecv[0][3],resultados_PCA_treecv[0][4],resultados_PCA_treecv[0][5]])

## MIM-MAX

In [ ]:
# ----------------------------------------
# Chama a função que roda o griSerachCV com os dados de treino para obter o melhor estimador
tic = time.time()

resultados_MIMMAX_CV = definir_parametros(X_train_MIMMAX, y_train_MIMMAX)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
# ----------------------------------------

# TREINANDO A ARVORE COM OS DADOS DE TREINO
dec_clf_cv.fit(X_train_MIMMAX, y_train_MIMMAX) 


# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_MIMMAX_treecv = roda_modelo('GRIDSEARCHCV + CROSSVALIDATION - DECISION TREE', 'MIM-MAX', X_train_MIMMAX, y_train_MIMMAX, X_test_MIMMAX, y_test_MIMMAX, dec_clf_cv)

In [ ]:
# Armazenando os dados
insert(metricas,['DECISION TREE REGRESSOR', 'GS + CV', 'MIM-MAX', resultados_MIMMAX_treecv[0][0],resultados_MIMMAX_treecv[0][1], resultados_MIMMAX_treecv[0][2], resultados_MIMMAX_treecv[0][3],resultados_MIMMAX_treecv[0][4],resultados_MIMMAX_treecv[0][5]])

## Z-SCORE

In [ ]:
# ----------------------------------------
# Chama a função que roda o griSerachCV com os dados de treino para obter o melhor estimador
tic = time.time()

resultados_scaler_CV = definir_parametros(X_train_scaler, y_train_scaler)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
# ----------------------------------------

# TREINANDO A ARVORE COM OS DADOS DE TREINO
dec_clf_cv.fit(X_train_scaler, y_train_scaler) 


# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_scaler_treecv = roda_modelo('GRIDSEARCHCV + CROSSVALIDATION - DECISION TREE', 'Z-Score', X_train_scaler, y_train_scaler, X_test_scaler, y_test_scaler, dec_clf_cv)

In [ ]:
# Armazenando os dados
insert(metricas,['DECISION TREE REGRESSOR', 'GS + CV', 'SCALER', resultados_scaler_treecv[0][0],resultados_scaler_treecv[0][1], resultados_scaler_treecv[0][2], resultados_scaler_treecv[0][3],resultados_scaler_treecv[0][4],resultados_scaler_treecv[0][5]])

# ROBUST

In [ ]:
# ----------------------------------------
# Chama a função que roda o griSerachCV com os dados de treino para obter o melhor estimador
tic = time.time()

resultados_robust_CV = definir_parametros(X_train_robust, y_train_robust)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
# ----------------------------------------

# TREINANDO A ARVORE COM OS DADOS DE TREINO
dec_clf_cv.fit(X_train_robust, y_train_robust) 


# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_robust_treecv = roda_modelo('GRIDSEARCHCV + CROSSVALIDATION - DECISION TREE', 'ROBUST', X_train_robust, y_train_robust, X_test_robust, y_test_robust, dec_clf_cv)

In [ ]:
# Armazenando os dados
insert(metricas,['DECISION TREE REGRESSOR', 'GS + CV', 'ROBUST', resultados_robust_treecv[0][0],resultados_robust_treecv[0][1], resultados_robust_treecv[0][2], resultados_robust_treecv[0][3],resultados_robust_treecv[0][4],resultados_robust_treecv[0][5]])

In [ ]:
metricas

# FIM DOS TESTES

In [ ]:
'''
# função que roda a árvore com os parâmetros definidos pelo gridserachCV

def insert(df, row):
    insert_loc = df.index.max()

    if np.isnan(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row
        
# CRIA A ARVORE - Os parâmetros definidos pelo GRIDSEARCHCV são iguais para os 4 tipos de dados (PCA, MIMMAX, Z-SCORE, ROBUST)
dec_clf_cv = tree.DecisionTreeClassifier(
    criterion='gini',
    max_depth= 5, # profundidade máxima da árvore
    min_samples_split= 2, # O número mínimo de amostras necessárias para dividir um nó interno
    min_samples_leaf= 80, # número mínimo de amostras necessárias para estar em um nó folha
    random_state= RANDOM_STATE, 
    splitter='best') 

def roda_arvore_cv (tipo, modelo, X_train, y_train, X_test, y_test):
    dec_clf_cv.fit(X_train, y_train) # TREINANDO A ARVORE COM OS DADOS DE TREINO

    desvio_y_original = y_test.std() # calculando o desvio do tareget original
    media_original = y_test.mean()

    y_pred_cv = dec_clf_cv.predict(X_test) # predizendo o target
    desvio_padrao_cv = y_pred_cv.std() # calculando o desvio do dado predito
    diferenca_cv = desvio_y_original - desvio_padrao_cv # calculando a diferença entre o desvio dos dados originais e do predito
    acuracia_cv = accuracy_score(y_test, y_pred_cv) #calculando a acurácia
    media_predita = y_pred_cv.mean()
    diferenca_media = media_original - media_predita
    print ('BASELINE', '|', modelo, '| Desvio_original:', desvio_y_original.round(3), '| Desvio padrão:', desvio_padrao_cv.round(3), '| Diferença:', diferenca_cv.round(3), '| Acurácia:', acuracia_cv.round(3)*100, ' | Media original:', media_original, ' | media_predita:', media_predita, ' | diferenca_media:', diferenca_media)

    # -----------------------------------------------
    # inserindo os dados no dataframe de metricas
    #insert(metricas, [tipo, modelo, desvio_y_original, desvio_padrao, diferenca, acuracia])
    # -----------------------------------------------
    return dec_clf_cv, y_pred_cv

    '''

### PCA

In [ ]:
'''
# Executa o GRIDSEARCHCV com os dados do PCA
tic = time.time()

resultados_PCA_CV = definir_parametros(X_train_PCA, y_train_PCA)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
'''

In [ ]:
'''
# RODAR A ÁRVORE COM OS PARAMETROS DEFINIDOS PELO GRIDSERACHCV
resultados_PCA_cv = roda_arvore_cv('ÁRVORE COM RESULTADO CV', 'PCA', X_train_PCA, y_train_PCA, X_test_PCA, y_test_PCA)
'''

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_PCA,resultados_PCA_cv[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_PCA_cv[0].classes_), index=(resultados_PCA_cv[0].classes_))
'''

### MIMMAX

In [ ]:

'''
# Executa o GRIDSEARCHCV com os dados do MIMMAX

tic = time.time()

resultados_MIMMAX_CV = definir_parametros(X_train_MIMMAX, y_train_MIMMAX)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
'''

In [ ]:
'''
# RODAR A ÁRVORE COM OS PARAMETROS DEFINIDOS PELO GRIDSERACHCV
resultados_MIMMAX_cv = roda_arvore_cv('COM GRIDSEARCHCV', 'MIMMAX', X_train_MIMMAX, y_train_MIMMAX, X_test_MIMMAX, y_test_MIMMAX)
'''

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_MIMMAX,resultados_MIMMAX_cv[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_MIMMAX_cv[0].classes_), index=(resultados_MIMMAX_cv[0].classes_))
'''

### Z-SCORE

In [ ]:
'''
tic = time.time()

resultados_scaler_CV = definir_parametros(X_train_scaler, y_train_scaler)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
'''

In [ ]:
#resultados_scaler_cv = roda_arvore_cv('BASELINE', 'Z-SCORE', X_train_scaler, y_train_scaler, X_test_scaler, y_test_scaler)

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_scaler,resultados_scaler_cv[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_scaler_cv[0].classes_), index=(resultados_scaler_cv[0].classes_))
'''

### ROBUST

In [ ]:
'''
tic = time.time()

resultados_robust_CV = definir_parametros(X_train_robust, y_train_robust)

tac = time.time()
tempo_que_passou = tac - tic
print("Tempo %.2f segundos" % tempo_que_passou)
'''

In [ ]:
#resultados_robust_cv = roda_arvore('BASELINE', 'Z-SCORE', X_train_robust, y_train_robust, X_test_robust, y_test_robust)

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_robust,resultados_robust_cv[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_robust_cv[0].classes_), index=(resultados_robust_cv[0].classes_))
'''

# MODELO 2 - RANDOM FOREST

In [ ]:
'''
def insert(df, row):
    insert_loc = df.index.max()

    if np.isnan(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row

randonForest = RandomForestRegressor()

def roda_random_forest (tipo, modelo, X_train, y_train, X_test, y_test):
    randonForest.fit(X_train, y_train) # TREINANDO A ARVORE COM OS DADOS DE TREINO

    desvio_y_original = y_test.std() # calculando o desvio do tareget original
    media_original = y_test.mean()

    y_pred = randonForest.predict(X_test) # predizendo o target
    desvio_padrao = y_pred.std() # calculando o desvio do dado predito
    diferenca = desvio_y_original - desvio_padrao # calculando a diferença entre o desvio dos dados originais e do predito
    media_predita = y_pred.mean()
    diferenca_media = media_original - media_predita
    #acuracia = accuracy_score(y_test, y_pred) #calculando a acurácia

    print ('BASELINE', '|', modelo, '| Desvio_original:', desvio_y_original.round(3), '| Desvio padrão:', desvio_padrao.round(3), '| Diferença:', diferenca.round(3), #'\n Acurácia:', acuracia.round(3)*100,
    ' \n Média_original', media_original.round(3), ' | Media predita:', media_predita.round(3), '| Diferença-média', diferenca_media.round(3))

    # -----------------------------------------------
    # inserindo os dados no dataframe de metricas
    #insert(metricas, [tipo, modelo, desvio_y_original, desvio_padrao, diferenca, acuracia])
    # -----------------------------------------------
    return randonForest, y_pred
    '''

# inicio - apagar depois de ajustar todo o código

In [ ]:
'''
# criando do dataframe para armazenar as métricas coletadas a cada execução do modelo
columns = ['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'] 
''''
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
metricas = pd.DataFrame(columns=columns)


def insert(df, row):
    insert_loc = df.index.max()

    if np.isnan(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row

        '''

In [ ]:
'''
import itertools



def roda_random_forest (tipo, nome_modelo, X_train, y_train, X_test, y_test, modelo):
    modelo.fit(X_train, y_train) # TREINANDO A ARVORE COM OS DADOS DE TREINO
    y_pred = modelo.predict(X_test) # predizendo o target
    
    print ("{} do {}".format(tipo, nome_modelo))
    print ("\n")

    classes = [0,1,2,3,4,5,6,7,8,9,10]
    resultados = mostra_resultados(y_test, y_pred, classes,tipo, nome_modelo)
    
    return resultados



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.grid(False)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# MOSTRA RESULTADOS
def mostra_resultados(y, y_pred, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.coolwarm):

    # Calcular os resultados
    cm = confusion_matrix(y, y_pred.round(0))
    FP = cm.sum(axis=0) - np.diag(cm)  # Falso Positivos
    FN = cm.sum(axis=1) - np.diag(cm)  # Falso Negativos
    TP = np.diag(cm)                   # Verdadeiro Positivo
    TN = cm.sum() - (FP + FN + TP)     # Verdadeiro Negativos
    TPR = TP / (TP + FN)  # Revocação    
    PPV = TP / (TP + FP)  # Precisao
    F1 = 2 * ((PPV * TPR) / (PPV + TPR)) # F1 Score
    ACC = (TP + TN) / (TP + FP + FN + TN)  # Acurácia

    desvio_y_test = y.std()
    desvio_y_pred = y_pred.std()
    diferenca_desvio = desvio_y_test - desvio_y_pred

    media_y_test = y.mean()
    media_y_prev = y_pred.mean()
    diferenca_media = media_y_test - media_y_prev

    # Exibe resultados
    print("    Verdadeiros Positivos:{}".format(TP))
    print("    Verdadeiros Negativos:{}".format(TN))
    print("    Falso Positivo:{}".format(FP))
    print("    Falso Negativo:{}".format(FN))
    
    print("    Revocação:{}".format(TPR))    
    print("    Precisao:{}".format(PPV))
    
    
    print("\n\n")

    print("    F1 Score:{}".format(F1.round(4)))
    print("    \nAcuracia:{}".format(ACC.round(4)*100))

    print("    \n Desvio \n")
    print("    y_tes:{}".format(desvio_y_test.round(4)*100))
    print("    y_pred:{}".format(desvio_y_pred.round(4)*100))
    print("    Diferença:{}".format(diferenca_desvio.round(4)*100))
    
    print("    \n Média \n")
    print("    y_tes:{}".format(media_y_test.round(4)*100))
    print("    y_pred:{}".format(media_y_prev.round(4)*100))
    print("    Diferença:{}".format(diferenca_media.round(4)*100))
    
    print("\n\n")

    plot_confusion_matrix(cm, classes)

    return desvio_y_test, desvio_y_pred, diferenca_desvio, media_y_test, media_y_prev, diferenca_media, y_pred
'''

# fim - apagar depois de ajustar todo o código

In [ ]:
# Criando o modelo
randonForest = RandomForestRegressor()

In [ ]:
randonForest

### PCA

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_PCA_randomForest = roda_modelo('BASELINE RANDON FOREST', 'PCA', X_train_PCA, y_train_PCA, X_test_PCA, y_test_PCA, randonForest)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['RANDOM FOREST REGRESSOR', 'BASELINE', 'PCA', resultados_PCA_randomForest[0][0],resultados_PCA_randomForest[0][1], resultados_PCA_randomForest[0][2], resultados_PCA_randomForest[0][3],resultados_PCA_randomForest[0][4],resultados_PCA_randomForest[0][5]])

### MIMMAX

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_MIMMAX_randomForest = roda_modelo('BASELINE RANDON FOREST', 'MIM-MAX', X_train_MIMMAX, y_train_MIMMAX, X_test_MIMMAX, y_test_MIMMAX, randonForest)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['RANDOM FOREST REGRESSOR', 'BASELINE', 'PCA', resultados_MIMMAX_randomForest[0][0],resultados_MIMMAX_randomForest[0][1], resultados_MIMMAX_randomForest[0][2], resultados_MIMMAX_randomForest[0][3],resultados_MIMMAX_randomForest[0][4],resultados_MIMMAX_randomForest[0][5]])

### Z-SCORE

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_scaler_randomForest = roda_modelo('BASELINE RANDON FOREST', 'SCALER', X_train_scaler, y_train_scaler, X_test_scaler, y_test_scaler, randonForest)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['RANDOM FOREST REGRESSOR', 'BASELINE', 'SCALER', resultados_scaler_randomForest[0][0],resultados_scaler_randomForest[0][1], resultados_scaler_randomForest[0][2], resultados_scaler_randomForest[0][3],resultados_scaler_randomForest[0][4],resultados_scaler_randomForest[0][5]])

### ROBUST

In [ ]:
# Executando o modelo, gerando os resuldos e plotando a matriz de confusão
resultados_robust_randomForest = roda_modelo('BASELINE RANDON FOREST', 'ROBUST', X_train_robust, y_train_robust, X_test_robust, y_test_robust, randonForest)

# Armazenando os dados
''''
insert(metricas,['nome_modelo', 'tipo_predicao', 'tipo_dado', 'desvio_original','desvio_pred', 'diferenca_desvio', 'media_original', 'media_predita', 'diferenca_media'])
# NOME_MODELO: 'DECISON TREE REGRESSOR', 'RANDOM FOREST REGRESSOR'
# TIPO_PREDICAO: BASELINE, CROSS VALIDATION + GRIDSEARCHCV
# TIPO_DADO: PCA, MIMMAX, SCALER, ROBUST
'''
insert(metricas,['RANDOM FOREST REGRESSOR', 'BASELINE', 'ROBUST', resultados_robust_randomForest[0][0],resultados_robust_randomForest[0][1], resultados_robust_randomForest[0][2], resultados_robust_randomForest[0][3],resultados_robust_randomForest[0][4],resultados_robust_randomForest[0][5]])

In [ ]:
metricas

# INICIO EXCLUIR

## RESULTADO DA PREDIÇÃO SEM GRIDSEARCH + CROSS VALIDATION

### PCA

In [ ]:
#resultados_PCA_randomForest = roda_random_forest('BASELINE RANDON FOREST', 'PCA', X_train_PCA, y_train_PCA, X_test_PCA, y_test_PCA)

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_robust,resultados_PCA_randomForest[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_PCA_randomForest[0].classes_), index=(resultados_PCA_randomForest[0].classes_))
111

### MINMAX

In [ ]:
#resultados_MIMMAX_randomForest = roda_random_forest('BASELINE RANDON FOREST', 'MIMMAX', X_train_MIMMAX, y_train_MIMMAX, X_test_MIMMAX, y_test_MIMMAX)

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_robust,resultados_MIMMAX_randomForest[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_MIMMAX_randomForest[0].classes_), index=(resultados_MIMMAX_randomForest[0].classes_))
'''

### Z-SCORE

In [ ]:
#resultados_scaler_randomForest = roda_random_forest('BASELINE RANDON FOREST', 'scaler', X_train_scaler, y_train_scaler, X_test_scaler, y_test_scaler)

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_robust,resultados_scaler_randomForest[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_scaler_randomForest[0].classes_), index=(resultados_scaler_randomForest[0].classes_))
'''

### ROBUST

In [ ]:
#resultados_robust_randomForest = roda_random_forest('BASELINE RANDON FOREST', 'robust', X_train_robust, y_train_robust, X_test_robust, y_test_robust)

In [ ]:
'''
# Compara o resultado do modelo com o resultado verdadeiro
cnf_matrix = confusion_matrix(y_test_robust,resultados_robust_randomForest[1])

# Constrói um Dataframe com a matriz de confusão apenas para ficar mais legível
pd.DataFrame(cnf_matrix, columns=(resultados_robust_randomForest[0].classes_), index=(resultados_robust_randomForest[0].classes_))
'''

In [ ]:
#mostra_resultados(y_test, y_pred, modelo.classes_)

# FIM EXCLUIR

## RESULTADO DA PREDIÇÃO COM GRIDSEARCH + CROSS VALIDATION

In [ ]:
'''
bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False

'''
'''
tree

                 'criterion':['gini', 'entropia'],
                 'max_depth': [5, 20, 100, 400, None], # profundidade máxima da árvore
                 'min_samples_split': [2, 50, 300, 1000], # O número mínimo de amostras necessárias para dividir um nó interno
                 'min_samples_leaf': [30, 120, 420, 820], # número mínimo de amostras necessárias para estar em um nó folha
                 'random_state': [RANDOM_STATE], 
                 'splitter': ['best']

'''

In [ ]:
# obtendo quais são os parâmetros disponíveis na árvore, para uso no grid-search
randonForest.get_params().keys()

In [ ]:
# PARAMETROS DO GRID SEARCH CV
parametros_grid_randomForest = {
    "n_estimators": [10, 50, 100, 200, 400], # O número de árvores na floresta.
    "max_depth": [5, 20, 100, 400, None], # profundidade máxima da árvore
    "max_features": ["auto", "sqrt", "log2"], # 
    "min_samples_split": [2, 50, 300, 1000], # número mínimo de amostras necessárias para dividir um nó interno
    "min_samples_leaf": [30, 120, 420, 820],  # número mínimo de amostras necessárias para estar em um nó folha
    "bootstrap": [True], # amostragem de dados
    'random_state': [RANDOM_STATE]
}

In [ ]:
# QUANTIDADE DE FOLDS 
n_cross_validation = 5

# USAR "ACURÁCIA" COMO CRITÉRIO PARA ENCONTRAR O MELHOR MODELO
criterio = "accuracy"

In [ ]:
radomForest_cv = GridSearchCV(RandomForestRegressor(),
                      param_grid=parametros_grid_randomForest, cv=n_cross_validation, verbose=10, scoring=criterio)

In [ ]:
# APLICANDO O TREINO
radomForest_cv.fit(X_train_PCA, y_train_PCA)

In [ ]:
# EXTRAINDO O MELHOR MODELO
melhor_modelo = radomForest_cv.best_estimator_

# PREDIÇÃO TESTE
y_pred = melhor_modelo.predict(X_test_PCA)

6. **Apresente (com visualizações) os resultados do seu modelo mostrando que ele é melhor do que um baseline não aleatório.**
    * Para o baseline, você pode escolher um modelo bem trivial mas não aleatório. Por exemplo, para um problema de classificação um baseline pode ser a classe mais presente caso o conjunto de dados seja desbalanceado.  Um modelo mais simples também pode ser utilizado como baseline, por exemplo, você escolheu um Random Forest Classifier, e comparou os resultados um Logistic Regression. Você pode até mesmo escolher um modelo de AutoML(como TPOT) como Baseline.

### 6) Conclusões **10%**

*Partindo das suas hipoteses e investigações, o que você consegue concluir? Suas hipoteses se concretizaram?*